In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import sys
project_root = os.path.abspath("..")  
sys.path.append(project_root)
from model.Transformer import TimeSeriesTransformer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load dataset
df = pd.read_csv(r'..\datasets\data\WTH.csv', index_col=0)
df

,Visibility,DryBulbFarenheit,DryBulbCelsius,WetBulbFarenheit,DewPointFarenheit,DewPointCelsius,RelativeHumidity,WindSpeed,WindDirection,StationPressure,Altimeter,WetBulbCelsius
date,,,,,,,,,,,,
1/1/2010 0:00,10.0,16,-9,13,7,-14,67,7,130,21.650000,30.35,-10.3
1/1/2010 1:00,10.0,16,-9,13,7,-14,67,5,150,21.640000,30.34,-10.3
1/1/2010 2:00,10.0,16,-9,13,7,-14,67,5,190,21.650000,30.35,-10.3
1/1/2010 3:00,10.0,16,-9,13,7,-14,67,7,180,21.650000,30.35,-10.3
1/1/2010 4:00,10.0,16,-9,14,9,-13,74,6,120,21.640000,30.34,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/2013 19:00,10.0,32,0,0,23,-5,0,0,0,21.478686,30.21,0.0
12/31/2013 20:00,7.0,30,-1,0,25,-4,0,5,110,21.478686,30.21,0.0
12/31/2013 21:00,5.0,30,-1,0,28,-2,0,0,0,21.478686,30.20,0.0


In [3]:
features = df.drop(columns=['WetBulbCelsius']).values
target = df['WetBulbCelsius'].values

In [4]:
print(features.shape, target.shape)

(35064, 11) (35064,)


In [5]:
feature_scaler = StandardScaler()
target_scaler = StandardScaler()
features = feature_scaler.fit_transform(features)
target = target_scaler.fit_transform(target.reshape(-1, 1)).flatten()

In [6]:
# Compute split indices
N = features.shape[0]
train_end = int(0.6 * N)
val_end = train_end + int(0.2 * N)

# Split the data (order is maintained; train DataLoader will shuffle)
X_train_np, y_train_np = features[:train_end], target[:train_end]
X_val_np, y_val_np = features[train_end:val_end], target[train_end:val_end]
X_test_np, y_test_np = features[val_end:], target[val_end:]

# Convert numpy arrays to PyTorch tensors
X_train = torch.tensor(X_train_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.float32)

X_val = torch.tensor(X_val_np, dtype=torch.float32)
y_val = torch.tensor(y_val_np, dtype=torch.float32)

X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_test = torch.tensor(y_test_np, dtype=torch.float32)

# Create TensorDatasets for each split
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoader for the training set with shuffle enabled
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("Train shapes:", X_train.shape, y_train.shape)
print("Validation shapes:", X_val.shape, y_val.shape)
print("Test shapes:", X_test.shape, y_test.shape)

Train shapes: torch.Size([21038, 11]) torch.Size([21038])
Validation shapes: torch.Size([7012, 11]) torch.Size([7012])
Test shapes: torch.Size([7014, 11]) torch.Size([7014])


In [7]:
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
model = TimeSeriesTransformer(input_dim=11, output_dim=1, seq_length=30).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
print("WTH DATASET")
print("---------------------------------------------------------")
epochs = 50
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_samples = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        # Ensure X_batch is of shape (batch_size, seq_length, input_dim).
        # If your dataset returns (batch_size, input_dim), expand it.
        if X_batch.ndim == 2:
            # Repeat along the sequence length dimension to match model.seq_length
            X_batch = X_batch.unsqueeze(1).repeat(1, model.seq_length, 1)
            
        optimizer.zero_grad()
        outputs = model(X_batch)
        # Squeeze output from (batch_size, 1) to (batch_size,) to compare with y_batch.
        loss = criterion(outputs.squeeze(-1), y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * X_batch.size(0)
        train_samples += X_batch.size(0)
    
    avg_train_loss = train_loss / train_samples

    # Evaluate on validation set
    model.eval()
    val_loss = 0.0
    val_samples = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            if X_batch.ndim == 2:
                X_batch = X_batch.unsqueeze(1).repeat(1, model.seq_length, 1)
            outputs = model(X_batch)
            loss = criterion(outputs.squeeze(-1), y_batch)
            val_loss += loss.item() * X_batch.size(0)
            val_samples += X_batch.size(0)
    avg_val_loss = val_loss / val_samples

    print(f"Epoch {epoch+1}/{epochs}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

# Evaluate on the test set after training
model.eval()
test_loss = 0.0
test_samples = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        if X_batch.ndim == 2:
            X_batch = X_batch.unsqueeze(1).repeat(1, model.seq_length, 1)
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(-1), y_batch)
        test_loss += loss.item() * X_batch.size(0)
        test_samples += X_batch.size(0)
avg_test_loss = test_loss / test_samples
print(f"Test Loss = {avg_test_loss:.4f}")


WTH DATASET
---------------------------------------------------------
Epoch 1/50: Train Loss = 0.0215, Val Loss = 0.0033
Epoch 2/50: Train Loss = 0.0058, Val Loss = 0.0026
Epoch 3/50: Train Loss = 0.0040, Val Loss = 0.0026
Epoch 4/50: Train Loss = 0.0036, Val Loss = 0.0009
Epoch 5/50: Train Loss = 0.0033, Val Loss = 0.0012
Epoch 6/50: Train Loss = 0.0027, Val Loss = 0.0043
Epoch 7/50: Train Loss = 0.0027, Val Loss = 0.0013
Epoch 8/50: Train Loss = 0.0024, Val Loss = 0.0032
Epoch 9/50: Train Loss = 0.0029, Val Loss = 0.0031
Epoch 10/50: Train Loss = 0.0023, Val Loss = 0.0013
Epoch 11/50: Train Loss = 0.0026, Val Loss = 0.0013
Epoch 12/50: Train Loss = 0.0025, Val Loss = 0.0010
Epoch 13/50: Train Loss = 0.0022, Val Loss = 0.0024
Epoch 14/50: Train Loss = 0.0023, Val Loss = 0.0051
Epoch 15/50: Train Loss = 0.0022, Val Loss = 0.0018
Epoch 16/50: Train Loss = 0.0020, Val Loss = 0.0016
Epoch 17/50: Train Loss = 0.0021, Val Loss = 0.0015
Epoch 18/50: Train Loss = 0.0020, Val Loss = 0.0013
Epo